# Ensuring the same transaction code has different economic impacts

In this training module we'll see how to use LUSID to perform the following task:

**<div align="center">As a data controller, I load transactions from two providers that use the same transaction code to signal different economic activity. I want to ensure, for each transaction, that LUSID applies the correct economic impact when generating holdings.</div>**

In [ ]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
from IPython.core.display import HTML
from datetime import datetime, timedelta
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusidjam import RefreshingToken
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.jupyter_tools import StopExecution
from finbourne_sdk_utils.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

In [ ]:
# Create a scope and code to segregate data in this module from other modules
module_scope = "FBNUniversity"
module_code = "Module-6-2"
print(f"'{module_scope}/{module_code}' scope and code created.")

## 1. Examining the source file

Imagine we source transactions from both UBS and Goldman Sachs:

* UBS uses a transaction code of `10` to signal an equity _purchase_.
* Goldman Sachs uses the same `10` transaction code to signal an equity _sale_.

We want to retain these codes in the imported data but ensure LUSID applies the correct economic impact when generating holdings.

In [ ]:
# Set variables to store useful values
transaction_code = "10"
transaction_source_ubs = "UBS"
transaction_source_gs = "GoldmanSachs"

# Read transactions from amalgamated stream into pandas dataframe
ubs_gs_df = pd.read_csv("data/same-code-ubs-gs.csv", keep_default_na = False)
ubs_gs_df.index += 1
display(ubs_gs_df)

## 2. Ensuring instruments are mastered correctly

It's possible the equity instrument in our transaction source file is already mastered in LUSID as part of the demonstration data, but for the avoidance of doubt we'll master it separately in a segregated custom instrument scope.

Note the `GBP` currency instrument, as per all currencies, is mastered out-of-the-box in the `default` instrument scope.

In [ ]:
# Obtain the LUSID Instruments API
instruments_api = api_factory.build(la.InstrumentsApi)
      
# Create a dictionary of instrument definitions for each equity in the transaction source file (ie. BP)
definitions = {
    security["instrument"]: lm.InstrumentDefinition(
            name = security["instrument"],
            identifiers = {
                "Figi": lm.InstrumentIdValue(value = security["figi"]),
            },
            definition = lm.Equity(
                instrument_type = "Equity",
                dom_ccy = security["currency"],
                identifiers = {}
            )
        )
    for index, security in ubs_gs_df.iterrows() if security["asset"] == "Equity"
}

# Upsert instruments to LUSID
upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    # Master the instruments in a custom scope
    scope = f"{module_scope}{module_code}",
)

# Transform API response to a dataframe and show internally-generated unique LUID for each mastered instrument
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
display(upsert_instruments_response_df[["name", "lusidInstrumentId"]])

## 3 Creating a suitable portfolio 
We register the custom instrument scope using the `instrument_scopes` field to ensure transactions upserted to the portfolio resolve first against instruments mastered in the custom scope, before falling back to the `default` scope.

In [ ]:
# Obtain the LUSID Transaction Portfolio API
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

# Create portfolio definition
portfolio_definition=lm.CreateTransactionPortfolioRequest(
    display_name="Training module 6.2 - Same transaction code, different impacts",
    code = module_code,
    base_currency = "GBP",
    # Must be before first transaction recorded
    created="2022-01-01T00:00:00Z",
    # Attempt to resolve transactions to instruments in the custom scope before falling back to the default scope
    instrument_scopes = [f"{module_scope}{module_code}"],
)

# Upsert portfolio to LUSID, making sure it's not already there
try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = module_scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    # Confirm success
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

## 4. Creating a custom transaction type for UBS to purchase an equity

We need to create a custom `10` transaction type for UBS to represent an equity _purchase_. 

The `type` and `source` values in the alias must be specific to UBS. We also set the `class` to something other than the default `Basic`, to ensure transaction types are reported properly.

The rest of the settings are replicated from the built-in `Buy` transaction type.

In [ ]:
# Obtain the LUSID Transaction Type Configuration API
transaction_config_api = api_factory.build(la.TransactionConfigurationApi)

transaction_type_definition = lm.TransactionTypeRequest(
    # Create an alias
    aliases = [
        lm.TransactionTypeAlias(
            # Specify the name of the transaction type
            type = transaction_code,
            description = "Equity purchase from UBS",
            # Set the transaction class to something other than 'Basic', which is used by the built-in types
            transaction_class = transaction_source_ubs,
            transaction_roles = "LongLonger"
        )
    ],
    # Replicate the movements from the built-in Buy transaction type
    movements = [
        lm.TransactionTypeMovement(
            movement_types = "StockMovement",
            side = "Side1",
            direction = 1,
            name = "Increase the equity holding by the number of units in the standard way"
        ),
        lm.TransactionTypeMovement(
            movement_types = "CashCommitment",
            side = "Side2",
            direction = -1,
            name = "Decrease cash position by total cost in the standard way"
        ),
    ]
)

# Create transaction type in LUSID    
try:
    response = transaction_config_api.set_transaction_type(
        # Specify the source (data provider) of the transaction type to be something other than the 'default' source
        source = transaction_source_ubs,
        # Specify the primary alias name (in this case, the name of the only alias in the transaction type)
        type = transaction_code,
        transaction_type_request = transaction_type_definition
    )
    # Confirm by retrieving the newly-created transaction type
    display(transaction_config_api.get_transaction_type(source = transaction_source_ubs, type = transaction_code))
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "TransactionTypeDuplication":
        logging.info(f"Transaction type {transaction_code} from {transaction_source_ubs} already exists.")

## 5. Creating a custom transaction type for Goldman Sachs to sell an equity

Now we need to create a custom `10` transaction type for Goldman Sachs to represent an equity _sale_. 

The `type`, `source` and `class` are specific to Goldman Sachs. The rest of the settings are replicated from the built-in `Sell` transaction type.

In [ ]:
# Obtain the LUSID Transaction Type Configuration API
transaction_config_api = api_factory.build(la.TransactionConfigurationApi)

transaction_type_definition = lm.TransactionTypeRequest(
    # Create an alias
    aliases = [
        lm.TransactionTypeAlias(
            # Specify the name of the transaction type
            type = transaction_code,
            description = "Equity sale from Goldman Sachs",
            # Set the transaction class to something other than 'Basic', which is used by the built-in types
            transaction_class = transaction_source_gs,
            transaction_roles = "LongShorter"
        )
    ],
    # Replicate the movements from the built-in Sell transaction type
    movements = [
        lm.TransactionTypeMovement(
            movement_types = "StockMovement",
            side = "Side1",
            direction = -1,
            name = "Decrease the equity holding by the number of units in the standard way"
        ),
        lm.TransactionTypeMovement(
            movement_types = "CashCommitment",
            side = "Side2",
            direction = 1,
            name = "Increase cash position by total cost in the standard way"
        ),
    ]
)

# Create transaction type in LUSID    
try:
    response = transaction_config_api.set_transaction_type(
        # Specify the source (data provider) of the transaction type to be something other than the 'default' source
        source = transaction_source_gs,
        # Specify the primary alias name (in this case, the name of the only alias in the transaction type)
        type = transaction_code,
        transaction_type_request = transaction_type_definition
    )
    # Confirm by retrieving the newly-created transaction type
    display(transaction_config_api.get_transaction_type(source = transaction_source_gs, type = transaction_code))
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "TransactionTypeDuplication":
        logging.info(f"Transaction type {transaction_code} from {transaction_source_gs} already exists.")

## 6. Loading transactions into the portfolio

For each equity transaction in the source file, we set the `source` field to either `UBS` or `GoldmanSachs` appropriately, to ensure it inherits the `10` transaction type with the correct economic impact.

Since the GBP transaction is using the built-in `FundsIn` type, we set its `source` field to `default`.

In [ ]:
# Create list of transactions to upsert
transactions = []

# For each row in dataframe
for index, txn in ubs_gs_df.iterrows():

    # Set variables based on whether instrument is cash or not
    if txn["asset"] == "Cash":
        # Set the unique instrument identifier to be a currency
        identifiers = {"Instrument/default/Currency": txn["currency"]}
        # Map the transaction code to a built-in transaction type in the 'default' source
        transaction_type_source = "default"
    else:
        # Set the unique instrument identifier to be a FIGI
        identifiers = {"Instrument/default/Figi": txn["figi"]}
        # Map the transaction code to a transaction type in the source specific to the data provider
        transaction_type_source = txn["source"]

    transactions.append(
        lm.TransactionRequest(
            transaction_id = str(txn["txn_id"]),
            # Set the transaction type to the transaction code from the data provider
            type = txn["txn_code"],
            instrument_identifiers = identifiers,
            transaction_date = txn["trade_date"],
            settlement_date = txn["settle_date"],
            units = txn["units"],
            transaction_price = lm.TransactionPrice(price = txn["price"], type="Price"),
            total_consideration = lm.CurrencyAndAmount(
                # Calculate cost on-the-fly
                amount = txn["units"] * txn["price"],
                currency = txn["currency"]
            ),
            # Set the source of the transaction type
            source = transaction_type_source
        )
    )

# Upsert transactions to LUSID
upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
    scope = module_scope, 
    code = module_code, 
    transaction_request = transactions
)

display(f"Transactions loaded at {str(upsert_transactions_response.version.as_at_date)}")
display(upsert_transactions_response)

## 7. Generating holdings

### 7.1 On the trade date
We can now call the `GetHoldings` API for end-of-day on the trade date (6 June 2022) to see the interim impact of our two `10` transaction types on holdings.

In [ ]:
# Get holdings for portfolio effective EOD 6 June 2022
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Specify an explicit date
    effective_at = "2022-06-06T17:00:00Z",
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
# get_holdings_response_df.drop(columns=[
#     "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)" ], inplace=True)
display(get_holdings_response_df)

### 7.2 Post-settlement date

If we call `GetHoldings` API without an explicit date, LUSID uses today's date so we can see the impact on holdings after the settlement date.

In [ ]:
# Get holdings for portfolio effective today
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = module_scope, 
    code = module_code,
    # Decorate the instrument name property onto holdings to make the API response more intuitive
    property_keys=["Instrument/default/Name"],
)

# Transform API response to a pandas dataframe and show it
get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
# Drop some noisy columns
# get_holdings_response_df.drop(columns=[
#     "sub_holding_keys", "cost_portfolio_ccy.currency", "currency", "SourcePortfolioId(default-Properties)", "SourcePortfolioScope(default-Properties)"], inplace=True)
display(get_holdings_response_df)